Install Libs

In [32]:
!pip install wandb --upgrade
!pip install pytorch-lightning
!pip install segmentation-models-pytorch
!pip install albumentations
!pip install python-dotenv
!pip install torchmetrics

Download Dataset

In [ ]:
# https://drive.google.com/file/d//view?usp=sharing
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mQCNp8dq499qnJI0YCc0hTVfmm4ppxO5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1mQCNp8dq499qnJI0YCc0hTVfmm4ppxO5" -O "download.tar.gz" && rm -rf /tmp/cookies.txt
!tar -xvf download.tar.gz

# Utils


# Dataset

In [1]:
import glob
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import albumentations as albu
from albumentations.pytorch import ToTensor
from typing import Optional, Tuple

In [2]:
class BeamPredictionDataset(Dataset):
    """Beam Prediction dataset."""

    def __init__(self, file_path: str,
                 label_file_path: str,
                 reshape: bool=False,
                 transforms: Optional[transforms.Compose] = None,
                 preprocessing_fn: Optional[transforms.Compose] = None) -> None:
        """
        Init the Dataset
        """
        self.file_path = file_path
        self.label_file_path = label_file_path
        
        self.data = np.load(file_path)
        self.data = self.data.transpose((1, 0))
        
        self.label = np.load(label_file_path)
#         pdb.set_trace()
        
        assert len(self.label) == len(self.data)
        # reshape is true
        # num_users x num_channels x num_antennas x real/imaginary
        if reshape:
            self.data = self.data.reshape((2, 4, 32, -1))
            self.data = self.data.transpose((3, 2, 1, 0))
            
        self.preprocessing_fn = preprocessing_fn
        self.transforms = transforms

    def __len__(self) -> int:
        """
        Returns the total length of dataset.
        """
        return len(self.data)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Gets an item from dataset
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        user_data = self.data[idx]
        
        if self.preprocessing_fn is not None:
            user_data = self.preprocessing_fn(user_data)
        
        if self.transforms is not None:
            user_data = self.transforms(user_data)
        
        label = self.label[idx]
        label = torch.Tensor(label).type(torch.int64) - 1
        return user_data, label
    
def transform(x: np.array) -> torch.Tensor:
    # mean normalize
    x -= x.mean()
    x /= x.std()
    x = torch.Tensor(x)
    return x

In [3]:
# example ds

ds = BeamPredictionDataset(
    file_path='./Dataset_-17.3375dB/-17.3375dB_inpTrain.npy',
    label_file_path='./Dataset_-17.3375dB/-17.3375dB_labelTrain.npy',
    transforms=transform
)
it = iter(ds)
out = next(it)
print(out)

(tensor([ 1.1344,  1.1642,  0.8118,  0.1093, -0.2929,  0.3619,  1.0790,  1.3874,
         1.2894,  1.1469,  1.3742,  0.6998, -0.4009,  0.3625,  0.8699,  1.1849,
         0.6786,  1.1753,  1.1628,  1.0550, -0.8943, -0.7125,  0.3022,  1.0900,
         0.8706,  1.3478,  1.5702,  0.9558, -1.6566, -0.7382,  0.0148,  0.6442,
         0.3017,  0.8905,  0.9791,  1.1644, -0.9876, -0.7922, -0.4629,  0.3374,
        -0.0516,  0.6355,  1.2137,  1.3104, -1.2456, -1.1153, -1.0029,  0.0712,
        -0.3255,  0.3647,  1.0288,  1.1176, -1.1510, -1.2242, -1.2269, -0.4542,
        -0.7970, -0.4256,  0.9413,  1.4918, -0.9748, -1.5398, -1.6161, -0.4450,
        -1.1006, -0.9013,  0.4272,  1.0205, -0.8424, -1.2304, -1.5746, -1.4980,
        -1.5050, -0.9478, -0.2943,  0.4626, -0.4176, -1.3170, -1.5857, -1.3130,
        -1.2471, -1.1989, -0.6621,  0.2399, -0.2827, -1.0135, -1.6343, -1.5665,
        -1.4390, -1.5518, -0.7836, -0.4450, -0.0606, -0.6873, -1.3370, -2.0944,
        -1.3866, -1.2450, -1.3194, -0.6

# Model

In [12]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pytorch_lightning as pl
import wandb
from argparse import ArgumentParser
import segmentation_models_pytorch as smp
from typing import Tuple

import torchmetrics 
import pdb

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class BeamClassifier(pl.LightningModule):

    def __init__(self, hparams) -> None:
        """
        Downloading Backbone and defining structure of model.
        """
        super().__init__()
        # args from argparser
        self.hparams = hparams
        in_ch = self.hparams.in_ch
        out_ch = self.hparams.out_ch
        
        self.model = nn.Sequential(
            nn.Linear(in_ch, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(512, out_ch),
        )
        
        self.acc_metric = torchmetrics.Accuracy()
        self.F1_metric = torchmetrics.F1(num_classes=self.hparams.out_ch)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """ Forward step of model.
        """
#         pdb.set_trace()
        x = self.model(x)
        return x

    def loss_fn(self, pred: torch.Tensor, y: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Loss function used for model"""
        y_sq = y.squeeze(-1)
        loss = F.cross_entropy(pred, y_sq)
        return loss
    
    def configure_optimizers(self) -> torch.optim:
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        opt = torch.optim.Adam(self.model.parameters(),
                               lr=self.hparams.learning_rate)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            opt, self.hparams.max_nb_epochs, self.hparams.learning_rate)
        self.lr_scheduler = lr_scheduler
#         lr_scheduler = None
        return [opt], [lr_scheduler]

    def train_dataloader(self) -> DataLoader:
        """Define the data loader for training data"""
        # REQUIRED
        return DataLoader(BeamPredictionDataset(
                                file_path='./Dataset_-17.3375dB/-17.3375dB_inpTrain.npy',
                                label_file_path='./Dataset_-17.3375dB/-17.3375dB_labelTrain.npy',
                                transforms=transform
                            ),
                          batch_size=self.hparams.batch_size,
                          shuffle=True)

    def training_step(self, batch: list, batch_idx: int) -> dict:
        """Backward step of model"""
        # REQUIRED
        x, y = batch
        pred = self.forward(x)

        loss = self.loss_fn(pred, y)
        
        pred = F.softmax(pred, dim=-1)
        y_sq = y.squeeze(-1)
        acc = self.acc_metric(pred, y_sq)
        f1 = self.F1_metric(pred, y_sq)
        
        if self.lr_scheduler is not None:
            lr = self.lr_scheduler.get_last_lr()[0]

        if(batch_idx % self.hparams.wandb_log_num_iter == 0):
            wandb.log({
                'train_loss': loss,
            })
            
        return {
            'loss': loss,
            'train_acc': acc,
            'train_f1': f1
        }
    
    def training_epoch_end(self, outputs: list) -> None:
        acc = self.acc_metric.compute()
        f1 = self.F1_metric.compute()
        
        if self.lr_scheduler is not None:
            lr = self.lr_scheduler.get_last_lr()[0]
        else:
            lr = self.hparams.learning_rate
        logs = {
            'lr': lr,
            'train_acc': acc,
            'train_f1': f1
        }
        wandb.log(logs)
        self.log_dict(logs)
    
    def val_dataloader(self) -> DataLoader:
        """Define the data loader for validation data"""
        # OPTIONAL
        return DataLoader(BeamPredictionDataset(
                                file_path='./Dataset_-17.3375dB/-17.3375dB_inpVal.npy',
                                label_file_path='./Dataset_-17.3375dB/-17.3375dB_labelVal.npy',
                                transforms=transform
                            ),
                          batch_size=self.hparams.batch_size)

    def validation_step(self, batch: list, batch_idx: torch.Tensor) -> dict:
        """Validation step to be carried out on validation data."""
        # REQUIRED
        # pdb.set_trace()
        x, y = batch
        
        pred = self.forward(x)

        loss = self.loss_fn(pred, y)
        
        pred = F.softmax(pred, dim=-1)
        y_sq = y.squeeze(-1)
        acc = self.acc_metric(pred, y_sq)
        f1 = self.F1_metric(pred, y_sq)

        return {
            'val_loss': loss,
            'val_acc': acc,
            'val_f1': f1
        }

    def validation_epoch_end(self, outputs: list) -> None:
        """Use results from each validation step to generate validation stats at epoch end"""
        val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_acc = self.acc_metric.compute()
        val_f1 = self.F1_metric.compute()
        
        logs = {
            'val_loss': val_loss,
            'val_acc': val_acc,
            'val_f1': val_f1
        }
        wandb.log(logs)
        self.log_dict(logs)

    def load_encoder_weights(self) -> None:
        """Loads encoder weights from ckpt"""
        ckpt = torch.load(self.hparams.encoder_ckpt_path)
        pretrained_dict = ckpt['state_dict']
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if (
            'encoder' in k) and (k in model_dict)}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

    def load_model_weights_from_ckpt(self) -> None:
        """Load model weights to model on cpu"""
        ckpt = torch.load(self.hparams.model_ckpt_path,
                          map_location=torch.device('cpu'))
        pretrained_dict = ckpt['state_dict']
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k,
                           v in pretrained_dict.items() if (k in model_dict)}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)
        
    def _get_learning_rate(self) -> float:
        i = 0
        for param_group in self._optimizer.param_groups:
            if i == 0:
                learning_rate = param_group["lr"]
            else:
                if learning_rate != param_group["lr"]:
                    raise ValueError(
                        "different param groups have different lr")
        return learning_rate

    @staticmethod
    def add_model_specific_args(parent_parser: ArgumentParser) -> ArgumentParser:
        """
        Specify the hyperparams for this LightningModule
        """
        # MODEL specific arguments
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', default=0.02, type=float)
        parser.add_argument('--batch_size', default=32, type=int)
        parser.add_argument('--in_ch', default=256, type=int)
        parser.add_argument('--out_ch', default=64, type=int)
        parser.add_argument('--max_nb_epochs', default=1, type=int)
        return parser

In [13]:
from argparse import ArgumentParser, Namespace
args_str = [
        # model related args
        '--max_nb_epochs=1',
        '--learning_rate=1e-3',
        '--batch_size=16',
        '--in_ch=256',
        '--out_ch=64',
]
parser = ArgumentParser(add_help=False)
parser = BeamClassifier.add_model_specific_args(parser)
args= parser.parse_args(args_str)

model = BeamClassifier(args)
train_dl = model.train_dataloader()
it = iter(train_dl)
x, y = next(it)


pred = model(x)
loss = model.loss_fn(pred, y)
print(loss)

tensor(4.0360, grad_fn=<NllLossBackward>)


# Trainer

In [14]:
args_str = ['--tpu_cores=0',
        '--progress_bar_refresh_rate=20',
        '--wandb_run_name=baseline',
        '--wandb_project_name=Beam Prediction',
        '--wandb_log_num_iter=1',
        '--gpus=0',
        # model related args
        '--max_nb_epochs=100',
        '--learning_rate=5e-3',
        '--batch_size=128',
        '--in_ch=256',
        '--out_ch=64',
    ]

In [15]:
from dotenv import load_dotenv

PROJECT_ROOT = os.path.dirname(os.path.abspath('.'))
load_dotenv(dotenv_path=os.path.join(PROJECT_ROOT, '.env'))

True

In [ ]:
from pytorch_lightning import Trainer, seed_everything
from argparse import ArgumentParser, Namespace
import wandb

# import pdb

parser = ArgumentParser(add_help=False)
parser.add_argument('-wandb_run_name',
                '--wandb_run_name',
                help='Name of Wandb Run',
                default='run',
                type=str)
parser.add_argument('-wandb_project_name',
                    '--wandb_project_name',
                    help='Wandb Project Name',
                    default='deep_dream',
                    type=str)
parser.add_argument('-model_ckpt_path',
                    '--model_ckpt_path',
                    help='Model Checkpoint Path',
                    default='./ckpts/model.ckpt',
                    type=str)
parser.add_argument('-wandb_log_num_iter',
                    '--wandb_log_num_iter',
                    help='After how many batches, we will log in training loop',
                    default=1,
                    type=int)
parser.add_argument('-init_ckpt',
                    '--init_ckpt',
                    help='Initial Ckpt',
                    default=None,
                    type=str)

def main(args):
    """Main function that will perform all the training"""
    # init module
    model = BeamClassifier(args)

    # Using Wandblogger so that we can log our results to wandb
    wandb.init(name=args.wandb_run_name,
               project=args.wandb_project_name,
               config=vars(args))
        
    wandb.watch(model)

    # most basic trainer, uses good defaults
    trainer = Trainer(logger=[], 
                      gpus=args.gpus, 
                      max_epochs=args.max_nb_epochs, 
                      resume_from_checkpoint=args.init_ckpt)
#     pdb.set_trace()
    trainer.fit(model)
    
    ckpt_path = os.path.join('./ckpt', f"{args.wandb_project_name}", f"{args.wandb_run_name}.ckpt")
    ckpt_base_path = os.path.dirname(ckpt_path)
    trainer.save_checkpoint(ckpt_path)
    wandb.save(ckpt_path)

    return model

if __name__ == '__main__':

    # auto add args from trainer
    parser = Trainer.add_argparse_args(parser)

    # give the module a chance to add own params
    # good practice to define LightningModule speficic params in the module
    parser = BeamClassifier.add_model_specific_args(parser)

    # parse params
    args= parser.parse_args(args_str)

    seed_everything(123)

    model = main(args)

Global seed set to 123


val_loss,1.45293
val_acc,0.23706
val_f1,0.23706
_runtime,121
_timestamp,1618651940
_step,3025
train_loss,1.80217
lr,0.001
train_acc,0.22442
train_f1,0.22442


val_loss,█▃▂▁▁▁
val_acc,▁▄▆▇▇█
val_f1,▁▄▆▇▇█
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▇▇▆▅▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▂▁▁▂▂▁▂▁
lr,▁▁▁▁▁
train_acc,▁▄▅▇█
train_f1,▁▄▅▇█


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name       | Type       | Params
------------------------------------------
0 | model      | Sequential | 1.2 M 
1 | acc_metric | Accuracy   | 0     
2 | F1_metric  | F1         | 0     
------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.881     Total estimated model params size (MB)


Epoch 0:  70%|███████   | 595/850 [00:27<00:11, 21.35it/s, loss=1.86, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 1:  70%|███████   | 595/850 [00:28<00:12, 20.72it/s, loss=1.71, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 2:  70%|███████   | 595/850 [00:29<00:12, 20.26it/s, loss=1.59, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 2:  96%|█████████▌| 816/850 [00:32<00:01, 25.22it/s, loss=1.59, v_num=]


Epoch 3:  70%|███████   | 595/850 [00:28<00:12, 20.58it/s, loss=1.51, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 4:  70%|███████   | 595/850 [00:29<00:12, 20.28it/s, loss=1.44, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 5:  70%|███████   | 595/850 [00:28<00:12, 20.54it/s, loss=1.46, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 5:  90%|█████████ | 768/850 [00:31<00:03, 24.01it/s, loss=1.46, v_num=]


Epoch 6:  70%|███████   | 595/850 [00:28<00:12, 20.52it/s, loss=1.42, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 7:  70%|███████   | 595/850 [00:28<00:12, 20.75it/s, loss=1.4, v_num=] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  70%|███████   | 595/850 [00:30<00:13, 19.25it/s, loss=1.35, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 8:  89%|████████▉ | 760/850 [00:33<00:03, 22.91it/s, loss=1.35, v_num=]


Epoch 9:  70%|███████   | 595/850 [00:29<00:12, 20.43it/s, loss=1.35, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 10:  70%|███████   | 595/850 [00:28<00:12, 20.58it/s, loss=1.33, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 11:  70%|███████   | 595/850 [00:28<00:12, 20.64it/s, loss=1.34, v_num=]
Validating: 0it [00:00, ?it/s]
Validating:  55%|█████▌    | 141/255 [00:02<00:01, 64.53it/s]


Epoch 12:  70%|███████   | 595/850 [00:28<00:12, 20.77it/s, loss=1.3, v_num=] 
Validating: 0it [00:00, ?it/s]
Epoch 13:  70%|███████   | 595/850 [00:29<00:12, 20.05it/s, loss=1.29, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 14:  70%|███████   | 595/850 [00:29<00:12, 20.49it/s, loss=1.31, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 14:  84%|████████▍ | 712/850 [00:30<00:06, 22.98it/s, loss=1.31, v_num=]


Epoch 15:  70%|███████   | 595/850 [00:29<00:12, 20.40it/s, loss=1.26, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 16:  64%|██████▍   | 543/850 [00:26<00:14, 20.61it/s, loss=1.27, v_num=]